In [ ]:
# # Create demo data in a duckdb
# from splink import splink_datasets
# import pandas as pd
# import duckdb as duckdb

# df = splink_datasets.fake_1000

# con = duckdb.connect('demo.db')

# con.sql("create table fake_data as from df")

# con.close()

In [ ]:
import duckdb as duckdb

con = duckdb.connect('data/demo.db')

con.sql('select * from fake_data limit 5').df()


CatalogException: Catalog Error: Table with name fake_data does not exist!
Did you mean "duckdb_databases"?

In [2]:
# import duckdb as duckdb
from splink import Linker, SettingsCreator, DuckDBAPI, block_on
from splink.exploratory import profile_columns

from splink.blocking_analysis import (
    cumulative_comparisons_to_be_scored_from_blocking_rules_chart,
)

import splink.comparison_library as cl

In [3]:
settings = SettingsCreator(

    link_type = "dedupe_only",
    
    comparisons = [
        # see https://moj-analytical-services.github.io/splink/api_docs/comparison_library.html
        
        cl.NameComparison("first_name"),
        cl.NameComparison("surname"),
        
        cl.LevenshteinAtThresholds("dob"),
        
        cl.ExactMatch("city").configure(term_frequency_adjustments = True),
        
        cl.EmailComparison("email")
    ],
    
    blocking_rules_to_generate_predictions = [
    
        block_on("first_name"),
        "l.surname = r.surname",  # alternatively, you can write rules in their SQL form
    
        block_on("substr(first_name, 1, 1)", "city"),
        block_on("substr(surname, 1, 1)", "city")
    
    ],
    
    retain_intermediate_calculation_columns = True,
    em_convergence = 0.01
)

In [4]:
db_api = DuckDBAPI(connection = con)

In [5]:
profile_columns(con.sql('select * exclude unique_id from fake_data'), db_api)

alt.VConcatChart(...)

In [6]:
cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables = con.sql('select * from fake_data'),
    blocking_rules = settings.blocking_rules_to_generate_predictions,
    db_api = db_api,
    link_type = "dedupe_only"
)

alt.Chart(...)

In [7]:
linker = Linker('fake_data', settings, db_api = db_api)

In [8]:
## Train lambda via rules that we think really yield matches

deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    "l.email = r.email"
]

# recall: estimated % of true matches captured by this set of rules
linker.training.estimate_probability_two_random_records_match(deterministic_rules, recall=0.6)

Probability two random records match is estimated to be  0.00389.
This means that amongst all possible pairwise record comparisons, one in 257.25 are expected to match.  With 499,500 total possible comparisons, we expect a total of around 1,941.67 matching pairs


In [9]:
linker.training.estimate_u_using_random_sampling(max_pairs=5e5)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [10]:
training_blocking_rule = "l.first_name = r.first_name and l.surname = r.surname"

training_session_fname_sname = (
    linker.training.estimate_parameters_using_expectation_maximisation(training_blocking_rule)
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.first_name = r.first_name and l.surname = r.surname

Parameter estimates will be made for the following comparison(s):
    - dob
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname

Level Jaro-Winkler >0.88 on username on comparison email not observed in dataset, unable to train m value

Iteration 1: Largest change in params was -0.538 in the m_probability of dob, level `Exact match on dob`
Iteration 2: Largest change in params was 0.0336 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.00962 in probability_two_random_records_match

EM converged after 3 iterations
m probability not trained for email - Jaro-Winkler >0.88 on username (comparison vector value: 1). This usually means the comparison level was never observed in the training d

In [11]:
training_blocking_rule = "l.dob = r.dob"

training_session_dob = linker.training.estimate_parameters_using_expectation_maximisation(
    training_blocking_rule
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.dob = r.dob

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob

Level Jaro-Winkler >0.88 on username on comparison email not observed in dataset, unable to train m value

Iteration 1: Largest change in params was -0.412 in the m_probability of surname, level `Exact match on surname`
Iteration 2: Largest change in params was 0.0891 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.0506 in the m_probability of first_name, level `All other comparisons`
Iteration 4: Largest change in params was 0.0174 in probability_two_random_records_match
Iteration 5: Largest change in params was 0.00715 in probability_two_random_records_match

EM converged after 5 iterations
m p

In [12]:
linker.visualisations.match_weights_chart()

/Users/ericmm/Documents/GitHub/rl-bootcamp/.venv/lib/python3.13/site-packages/altair/vegalite/v6/api.py:4124: UserWarning: Automatically deduplicated selection parameter with identical configuration. If you want independent parameters, explicitly name them differently (e.g., name='param1', name='param2'). See https://github.com/vega/altair/issues/3891
  return _tp.from_dict(dct, validate=validate)


alt.VConcatChart(...)

In [13]:
linker.visualisations.m_u_parameters_chart()

alt.HConcatChart(...)

In [14]:
linker.visualisations.parameter_estimate_comparisons_chart()

alt.Chart(...)

In [15]:
linker.evaluation.unlinkables_chart()

alt.LayerChart(...)

In [16]:
settings = linker.misc.save_model_to_json(
    "demo_model.json", overwrite=True
)

In [17]:
## use higher value if output data is likely large
predictions = linker.inference.predict(threshold_match_probability = 0.1)

Blocking time: 0.01 seconds
Predict time: 0.10 seconds

 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'email':
    m values not fully trained


In [18]:
con.sql(f'from {predictions.physical_name} limit 5').df()

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,tf_first_name_l,tf_first_name_r,bf_first_name,...,bf_city,bf_tf_adj_city,email_l,email_r,gamma_email,tf_email_l,tf_email_r,bf_email,bf_tf_adj_email,match_key
0,11.398783,0.999630,26,29,Thomas,Thomas,4,0.006017,0.006017,85.446306,...,1.000000,1.0,gabriel.t54@nnichls.info,NaN,-1,0.001267,NaN,1.000000,1.000000,0
1,11.398783,0.999630,28,29,Thomas,Thomas,4,0.006017,0.006017,85.446306,...,1.000000,1.0,gabriel.t54@nichols.info,NaN,-1,0.002535,NaN,1.000000,1.000000,0
2,0.708579,0.620377,37,38,Theodore,Theodore,4,0.012034,0.012034,85.446306,...,0.460274,1.0,t.m39@brooks-sawyer.com,t.m39@brooks-sawyer.com,4,0.006337,0.006337,252.693111,0.346193,0
3,8.744664,0.997674,47,50,Erin,Erin,4,0.004813,0.004813,85.446306,...,1.000000,1.0,e.rogers3@hopkins.org,NaN,-1,0.002535,NaN,1.000000,1.000000,0
4,15.098595,0.999971,63,67,Adam,Adam,4,0.007220,0.007220,85.446306,...,1.000000,1.0,ajones6@cortez-wilcox.com,ajones6@cortez-wilcox.com,4,0.005070,0.005070,252.693111,0.432741,0


In [19]:
clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(predictions, threshold_match_probability = 0.85)

con.execute(f'from {clusters.physical_name} limit 5').df()

Completed iteration 1, num edges remaining to process: 290
Completed iteration 2, num edges remaining to process: 4
Completed iteration 3, num edges remaining to process: 0


,cluster_id,unique_id,first_name,surname,dob,city,email,cluster
0,0,2,Rob,Allen,1971-06-24,London,roberta25@smith.net,0
1,8,9,Evie,Dean,2015-03-03,Pootsmruth,evihd56@earris-bailey.net,3
2,12,12,Hall,Isla,1985-02-07,London,islahall97@gutierrez-bennett.com,4
3,14,14,Oliver,Griffiths,1991-10-26,Lunton,o.griffiths90@reyes-coleman.com,5
4,19,19,Rowe,Caleb,1992-12-20,Lvpreool,calebr@thompson.org,8


In [20]:
cluster_count = []

for i in [0.7, 0.75, 0.8, 0.85, 0.9, 0.95]:
    clusters = linker.clustering. cluster_pairwise_predictions_at_threshold(predictions, threshold_match_probability = i)
    con.execute(f'select count(distinct(cluster_id)) from {clusters.physical_name}')
    c = con.fetchone()[0]
    cluster_count.append(c)    

cluster_count

Completed iteration 1, num edges remaining to process: 276
Completed iteration 2, num edges remaining to process: 2
Completed iteration 3, num edges remaining to process: 0
Completed iteration 1, num edges remaining to process: 276
Completed iteration 2, num edges remaining to process: 4
Completed iteration 3, num edges remaining to process: 0
Completed iteration 1, num edges remaining to process: 286
Completed iteration 2, num edges remaining to process: 4
Completed iteration 3, num edges remaining to process: 0
Completed iteration 1, num edges remaining to process: 290
Completed iteration 2, num edges remaining to process: 4
Completed iteration 3, num edges remaining to process: 0
Completed iteration 1, num edges remaining to process: 276
Completed iteration 2, num edges remaining to process: 2
Completed iteration 3, num edges remaining to process: 0
Completed iteration 1, num edges remaining to process: 276
Completed iteration 2, num edges remaining to process: 2
Completed iteration

[425, 428, 429, 431, 440, 456]

In [21]:
con.close()